<a href="https://colab.research.google.com/github/pallavmarch/WebScraping_PsychologyToday/blob/main/psychologytoday_webscraping_dataset_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

!pip install pgeocode
import pgeocode

pd.set_option("display.max_columns", None)
df=pd.read_csv("/content/therapists_profiles_data_Texas.csv",encoding='latin-1')

In [99]:
df[df['Name'] == 'Kathy Lee Siepker']

,Profile URL,Name,Title,Location,State,ZIP Code,Phone,Individual Sessions ($),Couple Sessions ($),Pay By,Insurance,Top Specialties,Expertise,Types of Therapy,Personal Statement,Verified by Psychology Today,In Practice (years),Membership,Certificate,Attended University,Major/Degree,Graduation Year,Qualifications,Endorsement Count,Endorsed By,Age,Participants,Communities,Religion,I also speak
5743,https://www.psychologytoday.com/us/therapists/...,Kathy Lee Siepker,"Clinical Social Work/Therapist,PhD,LCSW,CDP","Flower Mound, TX 75028(214) 692-2076",TX,75028,(214) 692-2076,$150,$150,"ACH Bank transfer, American Express, Discover,...",Aetna | BlueCross and BlueShield | Carelon Beh...,Anxiety | Depression | Dementia,Career Counseling | Coping Skills | Emotional ...,Coaching | Cognitive Behavioral (CBT) | Cultur...,"I am a licensed clinical social worker, therap...",Yes,4,Not Found,National Council of Certified Dementia Practit...,The Ohio State University (PhD); Wash U (MSW),Social Work,2014,Verified byPsychology TodayLicensed by State o...,1,Jamie English - Clinical Social Work/Therapist...,Adults | Elders (65+),Individuals | Couples | Family,Bisexual Allied | Gay Allied | HIV / AIDS Alli...,Not Found,Korean


In [100]:
print(len(df[df['Location'].isnull()]))
df.dropna(subset=['Location'], inplace=True)
print(len(df[df['Location'].isnull()]))

2
0


## Adding columns

In [101]:
df['Age'] = df['Age'].replace({'Elders (65+)': 'Elders'})
df['Dementia'] = df['Expertise'].str.lower().str.contains('dementia') | df['Top Specialties'].str.lower().str.contains('dementia')
df['Autism'] = df['Expertise'].str.lower().str.contains('autism') | df['Top Specialties'].str.lower().str.contains('autism')
df['Depression'] = df['Expertise'].str.lower().str.contains('depression') | df['Top Specialties'].str.lower().str.contains('depression')
df['Elders (65+)']=df['Age'].str.lower().str.contains('elders')
df['ZIP Code']=df['ZIP Code'].astype(int)

In [102]:
df[df['Dementia'] == True][['Dementia', 'Elders (65+)']].value_counts()

Dementia  Elders (65+)
True      True            180
          False            35
Name: count, dtype: int64

## 1

In [103]:
# Initialize the geocoder for US ZIP codes
nomi = pgeocode.Nominatim('us')

# Function to fetch city and county from ZIP Code
def get_location_info(zip_code):
    location = nomi.query_postal_code(zip_code)
    return pd.Series([location['place_name'], location['county_name']])

df[['City', 'County']] = df['ZIP Code'].astype(str).apply(get_location_info)

In [104]:
df

,Profile URL,Name,Title,Location,State,ZIP Code,Phone,Individual Sessions ($),Couple Sessions ($),Pay By,Insurance,Top Specialties,Expertise,Types of Therapy,Personal Statement,Verified by Psychology Today,In Practice (years),Membership,Certificate,Attended University,Major/Degree,Graduation Year,Qualifications,Endorsement Count,Endorsed By,Age,Participants,Communities,Religion,I also speak,Dementia,Autism,Depression,Elders (65+),City,County
0,https://www.psychologytoday.com/us/therapists/...,Mario Christian Salgado,"Licensed Professional Counselor,MA,LPC","San Antonio, TX 78232(830) 988-4854",TX,78232,(830) 988-4854,$140,Not Found,"ACH Bank transfer, American Express, Discover,...",Aetna | BlueCross and BlueShield | Cigna and E...,Not Found,Anger Management | Anxiety | Body Positivity |...,Art Therapy | Cognitive Behavioral (CBT) | Exp...,"Hello, my name is Mario! I want you all to get...",Yes,4,Not Found,Not Found,Texas A&M University,MA - Clinical Mental Health Counseling,2020,Verified byPsychology TodayLicensed by State o...,2,"Kayla Davis - Licensed Professional Counselor,...",Children (6 to 10) | Preteen | Teen | Adults,Individuals,Not Found,Not Found,Not Found,False,False,True,False,San Antonio,Bexar
1,https://www.psychologytoday.com/us/therapists/...,Denis 'Woodja' Flanigan,"Psychologist,PhD","2211 Norfolk StSuite 206Houston, TX 77098(713)...",TX,77098,(713) 999-3640,$215,$215,"American Express, Cash, Check, Discover, Healt...",Out of Network,Grief | Anxiety | Relationship Issues,Bisexual | Career Counseling | Codependency | ...,Cognitive Behavioral (CBT) | Culturally Sensit...,We can feel the most alone when we are hurting...,Yes,Not Found,Not Found,Not Found,University of Florida,Not Found,2011,Verified byPsychology TodayExpiredLicensed by ...,0,Not Found,Teen | Adults,Individuals | Couples,Bisexual Allied | Gay Allied | HIV / AIDS Alli...,Other Spiritual or Religious Affiliations,Not Found,False,False,True,False,Houston,Harris
2,https://www.psychologytoday.com/us/therapists/...,Carol Burnett-Williams,"Licensed Professional Counselor,LPC(she, her)","Houston, TX 77082(346) 512-4081",TX,77082,(346) 512-4081,Not Found,Not Found,Not Found,Aetna | BlueCross and BlueShield | Cigna and E...,Anxiety | Coping Skills | Depression,ADHD | Alcohol Use | Anger Management | Behavi...,Acceptance and Commitment (ACT) | Child/Parent...,"My name is Carol B. Williams, and I am a licen...",Yes,Not Found,Not Found,Not Found,Not Found,Not Found,Not Found,Verified byPsychology TodayLicensed by State o...,0,Not Found,Preteen | Teen | Adults,Individuals | Couples | Family,Bisexual Allied | Gay Allied | Intersex Allied...,Not Found,Not Found,False,False,True,False,Houston,Harris
3,https://www.psychologytoday.com/us/therapists/...,"Wenona Andress, LCSW-S","Clinical Social Work/Therapist,LCSW","Wenona Andress, LCSWLaguna Vista, TX 78578(806...",TX,78578,(806) 474-5787,$60,$80,"Cash, Discover, Health Savings Account, Master...",Aetna | BlueCross and BlueShield | Cigna and E...,Chronic Pain | Mood Disorders | Stress,Addiction | Addictive Family Systems | ADHD | ...,Christian Counseling | Clinical Supervision an...,Do you have chronic pain and wonder if you can...,Yes,14,Not Found,University of Massachusetts Center-Tobacco Res...,University of Houston Graduate School of Socia...,Not Found,2005,Verified byPsychology TodayExpiredLicensed by ...,1,Robert Brown Fortney - Licensed Professional C...,Adults | Elders (65+),Individuals | Family,Single Mother,Christian,Not Found,False,False,True,True,Port Isabel,Cameron
4,https://www.psychologytoday.com/us/therapists/...,Clarissa Anahi Cantu,"Licensed Professional Counselor,MS,LPC,LPCC","Austin, TX 78701(512) 640-0239",TX,78701,(512) 640-0239,$150,$170,"Mastercard, Paypal, Venmo, Visa, Zelle",Aetna | BlueCross and BlueShield | Cigna and E...,Depression | Anxiety | Grief,Anger Management | Behavioral Issues | Bisexua...,Adlerian | Cognitive Behavioral (CBT) | Compas...,"Hi there, I am Clarissa! I know it take

### DEMENTIA

In [105]:
Dementia=df[df['Dementia']==True]

Dementia=Dementia[['Name','Title', 'Location', 'City', 'County','ZIP Code',
       'Individual Sessions ($)', 'Couple Sessions ($)',
       'Insurance', 'Top Specialties', 'Expertise', 'Types of Therapy',
       'In Practice (years)', 'Membership', 'Certificate',
       'Major/Degree', 'Graduation Year', 'I also speak', 'Dementia','Elders (65+)']]

Dementia['Individual Sessions ($)'] = Dementia['Individual Sessions ($)'].replace({'Not Found': '0'}).str.replace('$', '').str.replace(' ', '').astype(float)
Dementia['Couple Sessions ($)'] = Dementia['Couple Sessions ($)'].replace({'Not Found': '0'}).str.replace('$', '').str.replace(' ', '').astype(float)
Dementia['In Practice (years)'] = Dementia['In Practice (years)'].replace({'Not Found': '0'}).astype(float)

In [106]:
pivot_table1 = pd.pivot_table(Dementia,    #[Dementia['In Practice (years)'] != 0],  # Filter out zeros
                             values=['Individual Sessions ($)', 'Couple Sessions ($)','In Practice (years)'],
                             #columns=['Top Specialties'],
                             columns=['Dementia'],
                             aggfunc=['mean', 'median','min','max',  'std'],
                            )


pivot_table1.style.background_gradient(cmap='Blues').format('{:.2f}')

,mean,median,min,max,std
Dementia,True,True,True,True,True
Couple Sessions ($),70.36,0.00,0.00,350.00,86.91
In Practice (years),8.96,6.00,0.00,41.00,10.35
Individual Sessions ($),109.66,125.00,0.00,250.00,69.08


However There are rows with blank Individual Sessions ($), couple Sessions ($) and In Practice (years) which are not considered 0 instead of blank

In [107]:
def transform_insurance(row):
  insurance = str(row).lower()
  if "not found" in insurance:
    return "No"
  elif insurance == "out of network":
    return "Out of Network"
  elif "medicare" in insurance and "medicaid" in insurance:
    return "Medicare|Medicaid"
  elif "medicare" in insurance:
    return "Medicare"
  elif "medicaid" in insurance:
    return "Medicaid"
  else:
    return "Yes"

Dementia['Insurance_Category'] = Dementia['Insurance'].apply(transform_insurance)


Dementia.insert(6, 'Insurance_Category', Dementia.pop('Insurance_Category'))
Dementia.head()

,Name,Title,Location,City,County,ZIP Code,Insurance_Category,Individual Sessions ($),Couple Sessions ($),Insurance,Top Specialties,Expertise,Types of Therapy,In Practice (years),Membership,Certificate,Major/Degree,Graduation Year,I also speak,Dementia,Elders (65+)
4,Clarissa Anahi Cantu,"Licensed Professional Counselor,MS,LPC,LPCC","Austin, TX 78701(512) 640-0239",Austin,Travis,78701,Yes,150.0,170.0,Aetna | BlueCross and BlueShield | Cigna and E...,Depression | Anxiety | Grief,Anger Management | Behavioral Issues | Bisexua...,Adlerian | Cognitive Behavioral (CBT) | Compas...,7.0,Not Found,EMDR Therapy Basic Training2020,Masters of Science in Clinical Mental Health C...,2018,Spanish,True,True
9,Clarissa Anahi Cantu,"Licensed Professional Counselor,MS,LPC,LPCC","Austin, TX 78701(512) 640-0239",Austin,Travis,78701,Yes,150.0,170.0,Aetna | BlueCross and BlueShield | Cigna and E...,Depression | Anxiety | Grief,Anger Management | Behavioral Issues | Bisexua...,Adlerian | Cognitive Behavioral (CBT) | Compas...,7.0,Not Found,EMDR Therapy Basic Training2020,Masters of Science in Clinical Mental Health C...,2018,Spanish,True,True
28,Vanessa Joy,"Licensed Professional Counselor,PsyD,LPC-S","1609 West 6th StreetSuite BAustin, TX 78703(73...",Austin,Travis,78703,Out of Network,195.0,230.0,Out of Network,Relationship Issues | Family Conflict | Life T...,Anxiety | Career Counseling | Chronic Illness ...,Attachment-based | CDWF (Certified Daring Way ...,0.0,Not Found,Registered Yoga Teacher - Yoga Alliance281089 ...,Not Found,Not Found,Not Found,True,True
44,Vanessa Garcia,"Clinical Social Work/Therapist,MSW,LCSW-S(she,...","Conroe, TX 77384(936) 259-2410",Conroe,Montgomery,77384,Yes,180.0,225.0,Aetna | BlueCross and BlueShield | Cigna and E...,Grief | Life Transitions | Stress,Anger Management | Anxiety | Behavioral Issues...,Cognitive Behavioral (CBT) | Culturally Sensit...,3.0,Not Found,EMDRIA International Association2025,Masters in Social Work,2017,Not Found,True,True
86,Jaime Nichols,"Clinical Social Work/Therapist,LCSW","12319 Willow Wild DriveAustin, TX 78758(512) 8...",Austin,Travis,78758,Yes,95.0,95.0,Aetna | BlueCross and BlueShield | Cigna and E...,Trauma and PTSD | Anxiety | Grief,Addiction | Alcohol Use | Ambiguous Loss | Chr...,Acceptance and Commitment (ACT) | Coaching | C...,9.0,Not Found,Not Found,Not Found,2015,Italian | Spanish,True,True


In [108]:
Dementia['Top Specialties'].value_counts()

,count
Top Specialties,
Not Found,19
Anxiety | Coping Skills | Depression,6
Anxiety | Depression | Trauma and PTSD,5
Depression | Anxiety | Grief,4
Anxiety | Depression | Relationship Issues,3
...,...
Depression | Anger Management | Chronic Illness,1
Trauma and PTSD | Relationship Issues | Anxiety,1
Mood Disorders | Oppositional Defiance (ODD) | Parenting,1


In [109]:
# prompt: if the word dementia is present in Dementia['Top Specialties'], display the row in

# Assuming the provided code is already executed and the 'Dementia' DataFrame is available.

for index, row in Dementia.iterrows():
    if 'dementia' in str(row['Top Specialties']).lower():
        print(row)

Name                                                            Jeremy Berry
Title                          Licensed Professional Counselor,PhD,LPC-S,NCC
Location                                      Belton, TX 76513(254) 615-4381
City                                                                  Belton
County                                                                  Bell
ZIP Code                                                               76513
Insurance_Category                                                        No
Individual Sessions ($)                                                125.0
Couple Sessions ($)                                                    150.0
Insurance                                                          Not Found
Top Specialties                                                     Dementia
Expertise                  Anxiety | Bisexual | Body Positivity | Coping ...
Types of Therapy           Acceptance and Commitment (ACT) | Adlerian | C...

In [ ]:
def transform_specialties(row):
  specialties = str(row).lower()
  if "not found" in specialties:
    return "No specialties"
  elif specialties == "dementia":
    return "dementia"
  elif "dementia" in specialties:
    return "Dementia with others"

Dementia['Specialties_Category'] = Dementia['Top Specialties'].apply(transform_specialties)


#Dementia.insert(6, 'Insurance_Category', Dementia.pop('Insurance_Category'))
#Dementia.head()

In [ ]:
Dementia[Dementia['Top Specialties'].str.contains("Dementia", na=False)]

,Title,Location,ZIP Code,Individual Sessions ($),Couple Sessions ($),Insurance,Insurance_Category,Top Specialties,Specialties_Category,Expertise,Types of Therapy,Personal Statement,Verified by Psychology Today,In Practice (years),Membership,Certificate,Major/Degree,Qualifications,I also speak,Dementia
7037,"Psychologist,PhD(she, her)","Wendy K. Ringe, PhDDallas, TX 75252(469) 444-2137",75252,225.0,0.0,Out of Network,Out of Network,Mood Disorders | Dementia | Relationship Issues,Dementia with others,Anxiety | Asperger's Syndrome | Behavioral Iss...,Cognitive Behavioral (CBT) | Cognitive Rehabil...,"My passion is helping people cope with, learn ...",Yes,0.0,Not Found,Not Found,Not Found,Verified byPsychology TodayLicensed by State o...,Not Found,True
